### Import Libraries

In [2]:
# Most code adapted from Gabriel Weindel at https://github.com/GWeindel/hmp
# https://github.com/GWeindel/hmp/blob/main/tutorials/1-Data_loading.ipynb
import numpy as np 
import pandas as pd
import xarray as xr
import hsmm_mvpy as hmp
from pathlib import Path

### Set up paths and participants

In [3]:
# Path where pre-processed EEG data can be found
eeg_data_path = Path('preprocessing/sat1/')
behavioral_data_path = eeg_data_path / 'behavior'

# Path where output will be saved
output_filename = 'data.nc'
output_path = Path('data/sat1/') / output_filename

# Change array indexing when you want to create a dataset consisting of a subset of the participants
subj_ids = [subj_id.name.split('.')[0] for subj_id in eeg_data_path.glob('*.csv')]
print(subj_ids)

['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025']


### Load data into epoched xarray

In [4]:
data = []

for participant in subj_ids:
    print(f'Loading participant {participant}')

    behavior = pd.read_csv(behavioral_data_path / f'{participant}-cnv-sat3_ET.csv', sep=';')
    
    # Rename categorical variables for clarity and transparency
    behavior['movement'] = behavior.apply(lambda row: 'stim_left' if row['movement'] == -1 else 
                                      ('stim_right' if row['movement'] == 1 else np.nan), axis=1)
    behavior['resp'] = behavior.apply(lambda row: 'resp_left' if row['resp'] == 1 else 
                                      ('resp_right' if row['resp'] == 2 else np.nan ), axis=1)
    
    # Merging together the experimental conditions info to have the format condition/stimulus/response
    behavior['trigger'] = behavior['cue'] + '/' +  behavior['movement'] + '/' +  behavior['resp']
    
    # Filtering reaction times below 300 and above 3000
    behavior['RT'] = behavior.apply(lambda row: 0 if row['RT'] < 300 else (
                0 if row['RT'] > 3000 else row['RT']), axis = 1)
    
    # Read EEG data
    eeg_data = pd.read_csv(eeg_data_path / f'{participant}.csv', index_col=0).reset_index(drop=True)
    # Drop any column that is not time, epoch, electrode name
    eeg_data = eeg_data.drop(columns='condition')
    # Rename FP1 and FP2 to align to montage convention
    eeg_data = eeg_data.rename(columns={'FP1':'Fp1', 'FP2':'Fp2'})

    # Use HMP function to cut off epochs to RT duration
    data.append(hmp.utils.parsing_epoched_eeg(data=eeg_data, rts=behavior['RT'], conditions=behavior['trigger'], sfreq=100))

# Store participant data into xarray
epoched_data = xr.concat(data, pd.Series(subj_ids, name='participant'), fill_value={'event':'', 'data': np.nan})

# Save xarray for use in HMP bump estimation
# epoched_data.to_netcdf(output_path)

Loading participant 0001
NaN present in condition array, removing associated epoch and RT ([0])
Totaling 185 valid trials
Loading participant 0002
Totaling 190 valid trials
Loading participant 0003
Totaling 185 valid trials
Loading participant 0004
Totaling 159 valid trials
Loading participant 0005
Totaling 195 valid trials
Loading participant 0006
Totaling 185 valid trials
Loading participant 0007
Totaling 186 valid trials
Loading participant 0008
Totaling 182 valid trials
Loading participant 0009
Totaling 190 valid trials
Loading participant 0010
Totaling 184 valid trials
Loading participant 0011
Totaling 185 valid trials
Loading participant 0012
Totaling 189 valid trials
Loading participant 0013
Totaling 181 valid trials
Loading participant 0014
Totaling 183 valid trials
Loading participant 0015
Totaling 180 valid trials
Loading participant 0016
Totaling 195 valid trials
Loading participant 0017
Totaling 176 valid trials
Loading participant 0018
Totaling 181 valid trials
Loading par

In [8]:
epoched_data

<xarray.Dataset>
Dimensions:      (epochs: 200, electrodes: 30, samples: 231, participant: 25)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * electrodes   (electrodes) <U3 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 224 225 226 227 228 229 230
  * participant  (participant) object '0001' '0002' '0003' ... '0024' '0025'
Data variables:
    data         (participant, epochs, electrodes, samples) float64 nan ... nan
    event        (participant, epochs) object '' ... 'AC/stim_right/resp_left'
Attributes:
    sfreq:    100
    offset:   0

In [13]:
epoched_data.sel(participant='0001', epochs=1).to_dataframe().to_csv('test.csv')